In [36]:
import pandas as pd
df1=pd.read_excel('UDJG_ADULT_CONS1.xls')
df2=pd.read_excel('UDJG_ADULT_CONS2.xls')
df3=pd.read_excel('UDJG_ADULT_CONS3.xls')
df4=pd.read_excel('UDJG_ADULT_CONS4.xls')
dataset2=pd.read_excel('UDJG_ADULT_FOODS.xls')
dataset3=pd.read_excel('UDJG_ADULT_SUBJ.xls')

dataset = pd.concat([df1, df2, df3, df4], axis=0, ignore_index=True)
#dataset = pd.read_excel('UDJG_ADULT_CONS1.xls')

In [37]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import os
import random
from collections import Counter
import numpy as np

In [38]:
columns_to_delete=['SURVEY', 'CONDAY', 'CONMONTH', 'CONYEAR', 'EXCEPTIONDAY', 'TIMEHOUR', 
                   'TIMEMINUTES', 'UNITMEAS', 'BRAND', 'ORFACETSCODE', 'ORFACETS', 'ENFACETS', 'PACKAGE', 'OFACETS', 'ORRECIPECODE', 
                   'ENRECIPEDESC', 'AMOUNTRECIPE', 'FPREP', 'FOODEXRCODE']
dataset=dataset.drop(columns=columns_to_delete, axis=1)


columns_to_delete2=['SURVEY', 'COMMENTSFOOD', 'ENERGY', 'WATER', 'FAT', 'CARB', 'PROTEINS', 'ALCOHOL', 'ENFOODNAME', 'FOODEXCODE']
dataset2=dataset2.drop(columns=columns_to_delete2, axis=1)

columns_to_delete3=['SURVEY', 'COUNTRY', 'BIRTHDAY', 'BIRTHMONTH', 'BIRTHYEAR', 'FANTYEAR', 'SWEIGHT', 'SHEIGHT', 'SANTDAY', 'SANTMONTH', 'SANTYEAR', 'MWEIGHT', 
                    'MHEIGHT', 'ENRGYINTAKE', 'UNOVREP', 'WF', 'SPECIALCON', 'SPECDIET', 'ACTIVITY', 'ETHNIC', 'FANTMONTH', 'FANTDAY',
                    'EDUCATION', 'COMMENTSSUBJECT']
dataset3=dataset3.drop(columns=columns_to_delete3, axis=1)

In [39]:
columns_to_check_duplicates = ['ORSUBCODE', 'DAY', 'WEEK', 'SEASON', 'MEAL', 'PLACE', 'EATSEQ']
dataset_no_duplicates = dataset.drop_duplicates(subset=columns_to_check_duplicates, keep='first')
dataset_no_duplicates = dataset_no_duplicates.dropna(subset=['ORRECIPEDESC'])

In [40]:
foodcode_to_recipe = dataset2.set_index('ORFOODCODE')['ORFOODNAME'].to_dict()

mask = pd.isna(dataset_no_duplicates['ORRECIPEDESC'])
dataset_no_duplicates.loc[mask, 'ORRECIPEDESC'] = dataset_no_duplicates.loc[mask, 'ORFOODCODE'].map(foodcode_to_recipe)

dataset_no_duplicates.insert(loc=11, column='GENDER', value='')
dataset_no_duplicates.insert(loc=12, column='AGE', value='')
dataset_no_duplicates.insert(loc=13, column='WEIGHT', value='')
dataset_no_duplicates.insert(loc=14, column='HEIGHT', value='')
dataset_no_duplicates.insert(loc=15, column='GEO', value='')


In [41]:
dataset_no_duplicates.head(10)

,ORSUBCODE,DAY,WEEK,SEASON,MEAL,PLACE,EATSEQ,ORRECIPEDESC,ORFOODCODE,AMOUNTFRAW,AMOUNTFCOOKED,GENDER,AGE,WEIGHT,HEIGHT,GEO
3,F001,1,W1,S5,M3,P3,1,ciorba de perisoare (cu vegeta),204.2.2.2.1,3.0,3.0,,,,,
15,F001,1,W1,S5,M3,P3,2,tocanita de ciuperci,204.2.1.15.0,25.0,20.0,,,,,
25,F001,1,W1,S5,M5,P3,1,mamaliga cu lapte,101.1.1.1.0,200.0,200.0,,,,,
29,F001,2,W2,S5,M1,P3,1,cafea cu lapte (fara zahar),414.1.1.0.0,100.0,100.0,,,,,
32,F001,2,W2,S5,M3,P3,1,ciorba de perisoare,414.0.0.0.1,100.0,100.0,,,,,
43,F001,2,W2,S5,M3,P3,2,piure de cartofi,204.2.1.30.0,284.0,213.0,,,,,
59,F001,3,W3,S5,M3,P3,1,pilaf de orez cu carne de pasare,414.1.1.0.0,120.0,12.0,,,,,
74,F001,3,W3,S5,M3,P3,2,clatite,206.2.1.0.0,25.0,25.0,,,,,
79,F001,3,W3,S5,M5,P3,1,orez cu lapte,101.1.1.1.0,200.0,100.0,,,,,
84,F001,3,W3,S5,M7,P3,2,chec,204.1.1.0.1,2.0,2.0,,,,,


In [42]:
gender_to_recipe = dataset3.set_index('ORSUBCODE').to_dict()['GENDER']
age_to_recipe = dataset3.set_index('ORSUBCODE').to_dict()['AGE']
weight_to_recipe = dataset3.set_index('ORSUBCODE').to_dict()['WEIGHT']
height_to_recipe = dataset3.set_index('ORSUBCODE').to_dict()['HEIGHT']
geo_to_recipe = dataset3.set_index('ORSUBCODE').to_dict()['GEO']

dataset_no_duplicates.loc[:, 'GENDER'] = dataset_no_duplicates['ORSUBCODE'].map(gender_to_recipe)
dataset_no_duplicates.loc[:, 'AGE'] = dataset_no_duplicates['ORSUBCODE'].map(age_to_recipe)
dataset_no_duplicates.loc[:, 'WEIGHT'] = dataset_no_duplicates['ORSUBCODE'].map(weight_to_recipe)
dataset_no_duplicates.loc[:, 'HEIGHT'] = dataset_no_duplicates['ORSUBCODE'].map(height_to_recipe)
dataset_no_duplicates.loc[:, 'GEO'] = dataset_no_duplicates['ORSUBCODE'].map(geo_to_recipe)


In [43]:
dataset_no_duplicates.loc[:, 'WEEK'] = dataset_no_duplicates['WEEK'].str.replace('W', '').astype(int)
dataset_no_duplicates.loc[:, 'SEASON'] = dataset_no_duplicates['SEASON'].str.replace('S', '').astype(int)
dataset_no_duplicates.loc[:, 'MEAL'] = dataset_no_duplicates['MEAL'].str.replace('M', '').astype(int)
dataset_no_duplicates.loc[:, 'PLACE'] = dataset_no_duplicates['PLACE'].str.replace('P', '').astype(int)
dataset_no_duplicates.loc[:, 'GENDER'] = dataset_no_duplicates['GENDER'].str.replace('G', '').astype(int)
dataset_no_duplicates.loc[:, 'GEO'] = dataset_no_duplicates['GEO'].str.replace('RO', '').astype(int)

In [44]:
columns_to_delete=['ORFOODCODE', 'ORSUBCODE']
dataset_no_duplicates=dataset_no_duplicates.drop(columns=columns_to_delete, axis=1)


In [45]:
le = LabelEncoder()
#dataset['ORRECIPEDESC'] = le.fit_transform(dataset['ORRECIPEDESC'])
dataset_no_duplicates['ORRECIPEDESC_encoded'] = le.fit_transform(dataset_no_duplicates['ORRECIPEDESC'])

In [46]:
dataset_no_duplicates.head()

,DAY,WEEK,SEASON,MEAL,PLACE,EATSEQ,ORRECIPEDESC,AMOUNTFRAW,AMOUNTFCOOKED,GENDER,AGE,WEIGHT,HEIGHT,GEO,ORRECIPEDESC_encoded
3,1,1,5,3,3,1,ciorba de perisoare (cu vegeta),3.0,3.0,2,34,64.0,164,126,165
15,1,1,5,3,3,2,tocanita de ciuperci,25.0,20.0,2,34,64.0,164,126,870
25,1,1,5,5,3,1,mamaliga cu lapte,200.0,200.0,2,34,64.0,164,126,370
29,2,2,5,1,3,1,cafea cu lapte (fara zahar),100.0,100.0,2,34,64.0,164,126,45
32,2,2,5,3,3,1,ciorba de perisoare,100.0,100.0,2,34,64.0,164,126,164


In [47]:
dataset_no_duplicates=dataset_no_duplicates.sample(frac=1, random_state=42)
dataset_no_duplicates.reset_index(drop=True, inplace=True)

In [48]:
interest = dataset_no_duplicates.iloc[:, [col for col in range(dataset_no_duplicates.shape[1]) if col != 6]].values

# standardize data (mean = 0, std dev = 1)
scaler = StandardScaler()
interest_z = scaler.fit_transform(interest)

In [49]:
# apply k-means
random.seed(2345)

#kmeans = KMeans(init="random", n_clusters=5, n_init=10, max_iter=300,random_state=42)
kmeans = KMeans(n_clusters=5)
kmeans.fit(interest_z)

kmeans.n_iter_
kmeans.cluster_centers_

dataset_no_duplicates['cluster'] = kmeans.labels_
Counter(dataset_no_duplicates['cluster'])

c:\Users\Silvian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Counter({1: 5149, 2: 4905, 3: 4618, 0: 4203, 4: 2783})

In [50]:
# Gruparea datelor și calcularea frecvenței fiecărei rețete în fiecare cluster și GEO
popular_recipes = dataset_no_duplicates.groupby(['cluster', 'GEO'])['ORRECIPEDESC'].value_counts()

# Obținerea top 5 rețete pentru fiecare combinație de cluster și GEO
top5_recipes_per_cluster_geo = popular_recipes.groupby(level=[0, 1]).nlargest(10).reset_index(level=[0, 1], drop=True)

# Afișarea rezultatelor
top5_recipes_per_cluster_geo.to_csv('statistica.csv')

PermissionError: [Errno 13] Permission denied: 'statistica.csv'

In [ ]:
dataset_no_duplicates['BMI'] = np.ceil((dataset_no_duplicates['WEIGHT'] / (dataset_no_duplicates['HEIGHT'] / 100) ** 2) * 10) / 10

In [ ]:
dataset_no_duplicates.head()

,DAY,WEEK,SEASON,MEAL,PLACE,EATSEQ,ORRECIPEDESC,AMOUNTFRAW,AMOUNTFCOOKED,GENDER,AGE,WEIGHT,HEIGHT,GEO,ORRECIPEDESC_encoded,cluster,BMI
0,2,2,3,5,1,1,ceafa de porc la gratar,157.000,110.000,1,43,100.0,185,223,313,3,29.3
1,6,6,2,7,1,2,cozonac,1.136,1.136,2,26,60.0,170,213,606,0,20.8
2,6,6,3,7,1,1,cola,500.000,500.000,2,53,90.0,165,424,514,0,33.1
3,7,7,2,3,1,1,supa,25.000,20.000,1,45,106.0,179,125,1652,3,33.1
4,6,6,3,5,1,2,rosie,111.000,111.000,2,46,54.0,160,115,1370,0,21.1
